Vector and Matrix Operations-
Design Parallel algorithm to
1. Add two large vectors
2. Multiply vector and matrix
3. Multiply two N x N arrays using n^2 processors

In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-ghubuay6
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-ghubuay6
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=1e35b09f6f0ad265907db97bc3d2fb1b7ebb3faa41527748137186d3c102c857
  Stored in directory: /tmp/pip-ephem-wheel-cache-hfv9sh8u/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
code="""#include<iostream>
#include<cstdlib>
using namespace std;

__global__ void vectorAdd(int *a, int *b, int *result, int n) {
    int tid = blockIdx.x*blockDim.x + threadIdx.x;
    if(tid <= n) {
        result[tid] = a[tid] + b[tid];
    }
}

void print_array(int *a, int N) {
    for(int i=0; i<N; i++) {
        cout<<"  "<<a[i];
    }
    cout<<endl;
}

void init_array(int *a, int N) {
    srand((unsigned)time(0));
    for(int i=0; i<N; i++) {
        a[i] = rand()%10 + 1;
    }
}
void init_array1(int *a, int N) {
   // srand((unsigned)time(0));
    for(int i=0; i<N; i++) {
        a[i] = rand()%10 + 1;
    }
}
int main() {
    int *a, *b, *c;
    int *a_dev, *b_dev, *c_dev;
    int n = 8;           //24
    
    a = (int*)malloc(n * sizeof(n));
    b = (int*)malloc(n * sizeof(n));
    c = (int*)malloc(n * sizeof(n));

    int size = n * sizeof(int);
    
    cudaMalloc(&a_dev, size);
    cudaMalloc(&b_dev, size);
    cudaMalloc(&c_dev, size);
    
    init_array(a, n);
    init_array1(b, n);
    
    print_array(a, n);
    print_array(b, n);
        
    //cudaEvent_t start, end;
    //cudaEventCreate(&start);
    //cudaEventCreate(&end);
    
    cudaMemcpy(a_dev, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(b_dev, b, size, cudaMemcpyHostToDevice);
    
    //int threads = 1024;
    //int blocks = (n+threads-1)/threads;
    
    //cudaEventRecord(start);
    
    //vectorAdd<<<blocks,threads>>>(a_dev, b_dev, c_dev, n);
    vectorAdd<<<1,1024>>>(a_dev, b_dev, c_dev, n);
    
    //cudaEventRecord(end);
    
    //cudaDeviceSynchronize();
       
    //float time = 0.0;
    //cudaEventElapsedTime(&time, start, end);
    
    cudaMemcpy(c, c_dev, size, cudaMemcpyDeviceToHost);
    
    cout<<"Results : "<<endl;
    print_array(c, n);
 
    //cout<<"Time elapsed : "<<time<<endl;
        
    cudaFree(a_dev);
    cudaFree(b_dev);
    cudaFree(c_dev);
        
    return 0;
}
"""

In [ ]:
t=open("vecAdd.cu","w")
t.write(code)
t.close()

In [ ]:
!nvcc vecAdd.cu

In [ ]:
!nvprof ./a.out

==365== NVPROF is profiling process 365, command: ./a.out
  7  7  6  1  10  4  4  1
  10  3  8  2  1  9  6  1
Results : 
  17  10  14  3  11  13  10  2
==365== Profiling application: ./a.out
==365== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   40.40%  3.9040us         2  1.9520us  1.5680us  2.3360us  [CUDA memcpy HtoD]
                   34.77%  3.3600us         1  3.3600us  3.3600us  3.3600us  vectorAdd(int*, int*, int*, int)
                   24.83%  2.4000us         1  2.4000us  2.4000us  2.4000us  [CUDA memcpy DtoH]
      API calls:   99.02%  119.79ms         3  39.930ms  4.9690us  119.78ms  cudaMalloc
                    0.38%  458.69us         1  458.69us  458.69us  458.69us  cuDeviceTotalMem
                    0.27%  325.21us        97  3.3520us     146ns  146.01us  cuDeviceGetAttribute
                    0.14%  171.63us         3  57.210us  18.566us  121.61us  cudaFree
                    0.09%  109.56

In [ ]:
code1="""
#include<iostream>
using namespace std;
__global__ void matrixVector(int *vec, int *mat, int *result, int n, int m)
{
    int tid = blockIdx.x*blockDim.x + threadIdx.x;
    int sum=0;
    
    if(tid <= n) {
        for(int i=0; i<n; i++) {
            sum += vec[i]*mat[(i*m) + tid];
        }
        result[tid] = sum;
    }
}

void init_array(int *a, int n) {
  srand((unsigned)time(0));
    for(int i=0; i<n; i++)
      a[i] = rand()%n + 1;
}

void init_matrix(int *a, int n, int m) {
  srand((unsigned)time(0));
    for(int i=0; i<n; i++) {
        for(int j=0; j<m; j++) {
            a[i*m + j] = rand()%n + 1;
        }
    }
}

void print_array(int *a, int n) {
    for(int i=0; i<n; i++) {
        cout<<"  "<<a[i];
    }
    cout<<endl;
}

void print_matrix(int *a, int n, int m) {
    for(int i=0; i<n; i++) {
        for(int j=0; j<m; j++)
          cout<<"  "<<a[i*m + j];
        cout<<endl;
    }
}

int main() {
    int *a, *b, *c;
    int *a_dev, *b_dev, *c_dev;
    
    int n = 3;
    int m = 4;
    
    a = new int[n];
    b = new int[n*m];
    c = new int[m];
    
    init_array(a, n);
    init_matrix(b, n, m);
        
    cout<<"Initial array : "<<endl;
    print_array(a, n);
    cout<<"Initial matrix : "<<endl;
    print_matrix(b, n, m);
    cout<<"Initial resultant array : "<<endl;
    print_array(c, m);
    cout<<endl;
    
    cudaMalloc(&a_dev, sizeof(int)*n);
    cudaMalloc(&b_dev, sizeof(int)*n*m);
    cudaMalloc(&c_dev, sizeof(int)*m);
    
    cudaMemcpy(a_dev, a, sizeof(int)*n, cudaMemcpyHostToDevice);
    cudaMemcpy(b_dev, b, sizeof(int)*n*m, cudaMemcpyHostToDevice);
    
    matrixVector<<<m/256+1, 256>>>(a_dev, b_dev, c_dev, n, m);
    
    cudaMemcpy(c, c_dev, sizeof(int)*m, cudaMemcpyDeviceToHost);
    
    cout<<"Results : "<<endl;
    print_array(c, m);
    
    cudaFree(a_dev);
    cudaFree(b_dev);
    cudaFree(c_dev);
    
    delete[] a;
    delete[] b;
    delete[] c;
    
    return 0;
}
"""

In [ ]:
t1=open("vecMat.cu","w")
t1.write(code1)
t1.close()

In [ ]:
!nvcc vecMat.cu

In [ ]:
!nvprof ./a.out

Initial array : 
  1  2  1
Initial matrix : 
  1  2  1  2
  3  2  1  3
  1  2  2  1
Initial resultant array : 
  0  0  0  0

==437== NVPROF is profiling process 437, command: ./a.out
Results : 
  8  8  5  9
==437== Profiling application: ./a.out
==437== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   41.57%  4.5760us         1  4.5760us  4.5760us  4.5760us  matrixVector(int*, int*, int*, int, int)
                   35.17%  3.8720us         2  1.9360us  1.5680us  2.3040us  [CUDA memcpy HtoD]
                   23.26%  2.5600us         1  2.5600us  2.5600us  2.5600us  [CUDA memcpy DtoH]
      API calls:   98.97%  119.69ms         3  39.898ms  5.2550us  119.68ms  cudaMalloc
                    0.44%  537.15us         1  537.15us  537.15us  537.15us  cuDeviceTotalMem
                    0.30%  365.54us        97  3.7680us     147ns  146.78us  cuDeviceGetAttribute
                    0.11%  137.07us         3  45.688us 

In [1]:
code2="""
#include<iostream>
using namespace std;
__global__
void matrixMultiplication(int *a, int *b, int *c, int m, int n, int k)
{
    int row = blockIdx.y*blockDim.y + threadIdx.y;
    int col = blockIdx.x*blockDim.x + threadIdx.x;
    int sum=0;
    if(col<k && row<m) {
      for(int j=0;j<n;j++)
      {
          sum += a[row*n+j] * b[j*k+col];
      }
      c[k*row+col]=sum;
    }
    
}

void init_result(int *a, int m, int k) {
    for(int i=0; i<m; i++) {
      for(int j=0; j<k; j++) {
        a[i*k + j] = 0;
      }
    }
}

void init_matrix(int *a, int n, int m) {
  srand((unsigned)time(0));
    for(int i=0; i<n; i++) {
      for(int j=0; j<m; j++) {
        a[i*m + j] = rand()%10 + 1;
      }
    }
}

void print_matrix(int *a, int n, int m) {
    for(int i=0; i<n; i++) {
      for(int j=0; j<m; j++) {
        cout<<"  "<<a[i*m + j];
      }
      cout<<endl;
    }
    cout<<endl;
}

int main()
{
    
    int *a,*b,*c;
    int *a_dev,*b_dev,*c_dev;
    int m=5, n=4, k=3;
    
    a = new int[m*n];
    b = new int[n*k];
    c = new int[m*k];
    
    init_matrix(a, m, n);
    init_matrix(b, n ,k);
    init_result(c, m, k);
    
    cout<<"Initial matrix : "<<endl;
    
    print_matrix(a, m, n);
    print_matrix(b, n, k);
   // print_matrix(c, m, k);
    
    cudaMalloc(&a_dev, sizeof(int)*m*n);
    cudaMalloc(&b_dev, sizeof(int)*n*k);
    cudaMalloc(&c_dev, sizeof(int)*m*k);
       
    cudaMemcpy(a_dev, a, sizeof(int)*m*n, cudaMemcpyHostToDevice);
    cudaMemcpy(b_dev, b, sizeof(int)*n*k, cudaMemcpyHostToDevice);
    
    dim3 dimGrid(1,1);
    dim3 dimBlock(16,16);
    
    matrixMultiplication<<<dimGrid, dimBlock>>>(a_dev,b_dev,c_dev, m, n, k);
     
    cudaMemcpy(c, c_dev, sizeof(int)*m*k, cudaMemcpyDeviceToHost);
    
    cout<<"Result : "<<endl;
    print_matrix(c, m, k);
    
    cudaFree(a_dev);
    cudaFree(b_dev);
    cudaFree(c_dev);
    
    delete[] a;
    delete[] b;
    delete[] c;
    
    return 0;
}
"""

In [2]:
t2=open("matMul.cu","w")
t2.write(code2)
t2.close()

In [3]:
!nvcc matMul.cu

In [4]:
!nvprof ./a.out

Initial matrix : 
  9  10  1  10
  2  10  9  2
  1  3  3  5
  6  7  5  3
  2  1  3  7

  9  10  1
  10  2  10
  9  2  1
  3  3  5

==161== NVPROF is profiling process 161, command: ./a.out
Result : 
  220  142  160
  205  64  121
  81  37  59
  178  93  96
  76  49  50

==161== Profiling application: ./a.out
==161== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   45.72%  5.4720us         1  5.4720us  5.4720us  5.4720us  matrixMultiplication(int*, int*, int*, int, int, int)
                   32.62%  3.9040us         2  1.9520us  1.6000us  2.3040us  [CUDA memcpy HtoD]
                   21.66%  2.5920us         1  2.5920us  2.5920us  2.5920us  [CUDA memcpy DtoH]
      API calls:   99.43%  210.37ms         3  70.125ms  4.9820us  210.36ms  cudaMalloc
                    0.24%  501.49us         1  501.49us  501.49us  501.49us  cuDeviceTotalMem
                    0.15%  326.61us        97  3.3670us     166ns  146.21us  